In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
from itertools import product, permutations

import numba
from numba import njit,vectorize, jit
from tqdm import tqdm

import Estimators

In [3]:
#@njit
sig2 = 1
def GaussianKernel(x,y):
    return np.exp(-np.linalg.norm(x-y)**2/(2*sig2))

In [4]:
def importation(): 
    X_train = []
    X_test = []
    Y_train = []
    for i in range(3): 
        xtrain = pd.read_csv('data/Xtr'+str(i)+'.csv',delimiter= ',', header= None)
        xtrain = xtrain.iloc[1:,1].to_numpy()
        X_train.append(xtrain)
    
        xtest = pd.read_csv('data/Xte'+str(i)+'.csv',delimiter= ',', header= None)
        xtest = xtest.iloc[1:,1]
        X_test.append(xtest)
    
        Y_train.append(pd.read_csv('data/Ytr'+str(i)+'.csv',delimiter= ',')['Bound'].to_numpy())
    return X_train,X_test,Y_train

X_train,X_test,Y_train = importation()

In [5]:
X_mat100_0 = pd.read_csv('data/Xtr0_mat100.csv',delimiter= ' ', header= None).values
Y_mat100_0 = pd.read_csv('data/Ytr0.csv',delimiter= ',')['Bound'].to_numpy()

In [6]:
X_traintest = []
for i in range(3): 
    X_traintest.append(np.concatenate((X_train[i],X_test[i])))


In [7]:
def to_Kernel_train(X, Kernel): 
    length = X.shape[0]
    mat_K = np.zeros((length,length))
    for i in range(length):
        x_i = np.squeeze(X[i])
        for j in range(i,length): 
            x_j = np.squeeze(X[j])
            value = Kernel(x_i,x_j)
            mat_K[i,j] = value
            mat_K[j,i] = value 
    return mat_K

#@njit 
def to_Kernel_test(Xtrain,Xtest,Kernel):
    length_train = Xtrain.shape[0]
    length_test = Xtest.shape[0]
    bimat_K = np.zeros((length_train,length_test))
    for i in range(length_train):
        x_i = np.squeeze(Xtrain[i])
        for j in range(length_test): 
            x_j = np.squeeze(Xtest[j])
            value = Kernel(x_i,x_j)
            bimat_K[i,j] = value
    return bimat_K

In [11]:
# Compute phi for Spectrum kernel
def phi_spectrum(x,k,U):
    """U: list of the sequences of size k to look for in x"""
    phi_spec = np.zeros(len(U))
    for j, u in enumerate(U):
        for i in range(len(x)-k+1):
            if x[i:i+k] == u:
                phi_spec[j] +=1
    return phi_spec

#Compute phi for Substring kernel
def l(i):
    return i[-1] - i[0] + 1

def I(k,n):
    I = set()
    for ele in permutations(range(0,n),k):
        I.add(tuple(sorted(list(ele))))
    return list(I)

def phi_substring(x,k,U,lamb=0.5): #fonctionne (testé avec l'exemple page 392)
    """U: list of the sequences of size k to look for in x"""
    phi = np.zeros(len(U))
    I_kn = I(k,len(x))
    for j, u in enumerate(U):
        for i in I_kn:
            x_i = "".join([x[idx]  for idx in i])
            if x_i==u:
                phi[j] += lamb**l(i)
    return phi


def make_dict_phi(X, phi,k):
    U = [''.join(letter) for letter in product('ACGT', repeat=k)]
    phi_dict = {seq:phi(seq,k,U) for seq in tqdm(X)}
    return phi_dict
    

# Compute K(x_i,x) 
def K_substring_2(x, y):
    """
    Compute K(x_i,x) with K a string kernel
    
    Parameters:
    Ne prend pas en argument X_train mais seulement phi_X_train(qui a été calculé précédement)
    et i .
    
    phi_X_train: list of phi(X_train) which is also needed to construct the matrix K 
    phi: the function phi
    k: hyperparameter of the spectrum kernel (size of sequences in U)
    """
    value = np.sum([dict_sub_traintest_2[str(x)] * dict_sub_traintest_2[str(y)]])  
    return value

def K_substring_4(x,y): 
    value = np.sum([dict_sub_traintest_4[str(x)] * dict_sub_traintest_4[str(y)]])  
    return value
    

def K_spectrum_3(x,y): 
    value = np.sum([dict_spectrum_traintest_3[str(x)] * dict_spectrum_traintest_3[str(y)]])  
    return value
    
def K_spectrum_4(x,y): 
    value = np.sum([dict_spectrum_traintest_4[str(x)] * dict_spectrum_traintest_4[str(y)]])  
    return value

def K_spectrum_5(x,y): 
    value = np.sum([dict_spectrum_traintest_5[str(x)] * dict_spectrum_traintest_5[str(y)]])  
    return value




In [10]:
##CONSTRUCTION DES DICTIONNAIRES. On fait ça pour eviter de les recalculer à chaque fois

#dict_spectrum_traintest_3 = make_dict_phi(X_traintest[0],phi_spectrum,3)

#dict_spectrum_traintest_4 = make_dict_phi(X_traintest[0],phi_spectrum,4)

#dict_spectrum_traintest_5 = make_dict_phi(X_traintest[0],phi_spectrum,5)

#dict_sub_traintest_2 = make_dict_phi(X_traintest[0], phi_substring,2)

#dict_sub_traintest_3 = make_dict_phi(X_traintest[0], phi_substring,3)

100%|██████████| 3000/3000 [00:31<00:00, 95.57it/s]


In [185]:
Kernel_train_Gaussian = to_Kernel_train(X_mat100_0,GaussianKernel)
Kernel_test_Gaussian = to_Kernel_test(X_mat100_0,X_mat100_0,GaussianKernel)

In [194]:
regressor_Gaussian = Estimators.KRR(Kernel= GaussianKernel,lam = 0)
regressor_Gaussian.fit(Kernel_train_Gaussian,Y_train[0])
prediction_Gaussian = regressor_Gaussian.predict(Kernel_test_Gaussian)

In [196]:
np.max(prediction_Gaussian -Y_train[0]) # on check si nos prédictions sont bonnes 

0

In [197]:
Kernel_train_spectrum = to_Kernel_train(X_train[0], K_spectrum_3)
Kernel_test_spectrum = to_Kernel_test(X_train[0], X_train[0],K_spectrum_3)

In [199]:
regressor_spectrum = Estimators.KRR(Kernel = K_spectrum_3,lam = 0)
regressor_spectrum.fit(Kernel_train_spectrum,Y_train[0])
predictions_spectrum = regressor_sprectrum.predict(Kernel_test_spectrum)

In [200]:
(predictions_spectrum-Y_train[0])[0:100]

array([ 0,  0, -1,  0,  0,  0,  0,  1,  0,  0,  0, -1,  0,  1, -1,  0,  0,
       -1, -1,  0,  1, -1,  0, -1,  0, -1, -1,  0,  0,  0,  0, -1,  0, -1,
       -1,  1,  0,  1,  0, -1,  0,  1,  1,  1,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  1, -1, -1,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  1,
        1, -1,  0,  0,  0, -1,  1, -1,  0,  0,  0,  0,  0,  0, -1,  0,  0,
        1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0])

In [12]:
Kernel_train_spectrum_5 = to_Kernel_train(X_train[0], K_spectrum_5)
Kernel_test_spectrum_5 = to_Kernel_test(X_train[0], X_train[0],K_spectrum_5)

In [14]:
regressor_spectrum_5 = Estimators.KRR(Kernel = K_spectrum_5,lam = 0)
regressor_spectrum_5.fit(Kernel_train_spectrum_5,Y_train[0])
predictions_spectrum_5 = regressor_spectrum_5.predict(Kernel_test_spectrum_5)

In [15]:
(predictions_spectrum_5-Y_train[0])[0:100]

array([ 1,  0,  0, -1, -1,  0,  0,  1,  1, -1,  1,  0, -1,  1,  0,  0, -1,
        0, -1,  0,  1, -1, -1,  0,  0,  0,  0,  0,  1,  0,  0,  0, -1, -1,
        0,  0,  1,  0,  0, -1, -1,  1,  0,  0,  1,  0,  0, -1,  0,  1, -1,
        1, -1,  1,  0,  0,  0,  0,  0,  0, -1,  1,  1,  0,  1,  0,  0,  0,
        1, -1,  0,  0,  0, -1,  0, -1,  0,  0,  0, -1, -1,  0, -1,  0,  0,
        0,  1,  0,  0,  0, -1, -1,  0,  1,  1, -1,  0,  0, -1,  1])

In [148]:
regressor_substring = Estimators.KRR(Kernel= K_substring_2,lam = 0)

In [152]:
regressor_substring.fit(X_train[0],Y_train[0])

In [209]:
Kernel_train_substring = to_Kernel_train(X_train[0], K_substring_2)
Kernel_test_substring = to_Kernel_test(X_train[0],X_train[0],K_substring_2)

In [212]:
regressor_substring = Estimators.KRR(Kernel = K_substring_2)
regressor_substring.fit(Kernel_train_substring,Y_train[0])
predictions_substring = regressor_substring.predict(Kernel_test_substring)
(predictions_substring-Y_train[0])[0:100]

array([ 0, -1, -1,  0,  0,  1,  0,  1,  0,  0,  0, -1,  0,  1, -1,  1, -1,
       -1, -1, -1,  1, -1,  0, -1,  0,  0,  0,  0,  1,  1,  0,  0,  0, -1,
       -1,  0,  0,  1,  0, -1, -1,  1,  1,  1,  0,  1,  1,  0,  0,  1,  0,
        0,  0,  0,  0, -1, -1,  0,  0,  0, -1,  1,  0,  1,  0,  1,  0,  1,
        1, -1,  0,  0,  0, -1,  1,  0,  0,  0,  0,  0, -1,  0,  0,  0, -1,
        0,  0,  0,  0, -1, -1,  0,  0,  0,  0,  0,  0, -1, -1,  0])

In [215]:
print(np.max(Kernel_train_substring-Kernel_test_substring))

0.0


In [146]:
prediction_Gaussian = regressor_Gaussian.predict(X_mat100_0)

In [147]:
(prediction_Gaussian-Y_mat100_0)[0:2000]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
#mettre le rademcher dans la classe estimator 